In [65]:
#!mkdir data

In [66]:
#!pip install --upgrade xlrd

In [67]:
# !pip install stop_words
# !pip install pymorphy2

In [68]:
import numpy as np
import keras
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking,Flatten
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from tensorflow import keras
from keras.losses import categorical_crossentropy
from string import punctuation
# from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping 

In [69]:
data = pd.read_excel(open('/content/data/отзывы за лето (1).xls', 'rb'))
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [70]:
data.drop('Date', axis=1, inplace=True)

In [71]:
data.head()

,Rating,Content
0,5,It just works!
1,4,В целом удобноное приложение...из минусов хотя...
2,5,Отлично все
3,5,Стал зависать на 1% работы антивируса. Дальше ...
4,5,"Очень удобно, работает быстро."


In [72]:
from sklearn.model_selection import train_test_split

In [73]:
target = data['Rating']
df = data.copy()
df.drop('Rating', axis=1, inplace=True)

In [74]:
X_train, data_x, y_train, data_y = train_test_split(df,target, train_size=0.8)
X_val, X_test, y_val, y_test = train_test_split(data_x, data_y) 

In [75]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [76]:
X_train['Content'] = X_train['Content'].apply(preprocess_text)
X_val['Content'] = X_val['Content'].apply(preprocess_text)
X_test['Content'] = X_test['Content'].apply(preprocess_text)

In [77]:
max_words = 3000
max_len = 110
num_classes = 1
epochs = 10
batch_size = 512
print_batch_n = 100

In [78]:
print(X_train.shape), print(y_train.shape)
print(X_val.shape), print(y_val.shape)
print(X_test.shape), print(y_test.shape)

(16527, 1)
(16527,)
(3099, 1)
(3099,)
(1033, 1)
(1033,)


(None, None)

In [79]:
train_corpus = " ".join(X_train["Content"])
train_corpus = train_corpus.lower()

In [80]:
train_corpus

'оперативно удобно хороший приложение поправить вход отпечаток палец удобный думать проблема телефон приложение втб24 госуслуга такой проблемнуть таки проблема приложение устраивать  оставлять запустить хвост использовать оперативка процессор батарейка исправт чтобня жрать ресурс телефон нравиться приложениеудобный удобный приложение работник банк корректный профессиональный работа телефон meizu рутанета темень сообщать модификация прошивка наличие рута оставить покой попытка проверка наличие рута дать опция принять риск пользователь хороший удобный программа палата кредит банк работать самый удобный приложение сравнение другой приложение банк норма удобно легко пользоваться свой счёт устраиваетспасибо окей создатель уволить отлично рекомендовать удобство удобно отличный приложение нравиться удобно  удобный быстрый сервис отличный приложение работать чётко единственный вопрос разработчикамкогда заплатить инет поискеневозможный найти домра название инна приложение довольный лишнегопять 

In [81]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [82]:
tokens[:8]

['оперативно',
 'удобно',
 'хороший',
 'приложение',
 'поправить',
 'вход',
 'отпечаток',
 'палец']

In [83]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [84]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [85]:
tokens_filtered_top[:8]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный']

In [86]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [87]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [88]:
X_train = np.asarray([text_to_sequence(text, max_len) for text in X_train["Content"]], dtype=np.int32)
X_test = np.asarray([text_to_sequence(text, max_len) for text in X_test["Content"]], dtype=np.int32)
X_val = np.asarray([text_to_sequence(text, max_len) for text in X_val["Content"]], dtype=np.int32)

In [89]:
X_train.shape

(16527, 110)

In [90]:
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [91]:
num_classes = 6
y_train = tf.keras.utils.to_categorical(y_train['Rating'], num_classes)
y_test = tf.keras.utils.to_categorical(y_test['Rating'], num_classes)

In [92]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 110, 128)          384000    
                                                                 
 conv1d_6 (Conv1D)           (None, 108, 128)          49280     
                                                                 
 activation_4 (Activation)   (None, 108, 128)          0         
                                                                 
 global_max_pooling1d_2 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_16 (Dense)            (None, 10)                1290      
                                                                 
 activation_5 (Activation)   (None, 10)                0         
                                                      

In [93]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [94]:
early_stopping=EarlyStopping(monitor='val_loss', patience=7)  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
30/30 [==============================] - 13s 417ms/step - loss: 1.3642 - accuracy: 0.7023 - val_loss: 1.0628 - val_accuracy: 0.7199
Epoch 2/10
30/30 [==============================] - 12s 408ms/step - loss: 0.9552 - accuracy: 0.7066 - val_loss: 0.7848 - val_accuracy: 0.7223
Epoch 3/10
30/30 [==============================] - 12s 408ms/step - loss: 0.7760 - accuracy: 0.7315 - val_loss: 0.7010 - val_accuracy: 0.7756
Epoch 4/10
30/30 [==============================] - 12s 405ms/step - loss: 0.6984 - accuracy: 0.7732 - val_loss: 0.6635 - val_accuracy: 0.7792
Epoch 5/10
30/30 [==============================] - 14s 455ms/step - loss: 0.6467 - accuracy: 0.7917 - val_loss: 0.6465 - val_accuracy: 0.7786
Epoch 6/10
30/30 [==============================] - 12s 407ms/step - loss: 0.6034 - accuracy: 0.8059 - val_loss: 0.6303 - val_accuracy: 0.7864
Epoch 7/10
30/30 [==============================] - 12s 407ms/step - loss: 0.5633 - accuracy: 0.8189 - val_loss: 0.6296 - val_accuracy: 0.7858

In [95]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('-'*100)
print(f'max_words: {max_words}')
print(f'max_len: {max_len}')
print('Test score:', score[0])
print('Test accuracy:', score[1])

3/3 [==============================] - 0s 63ms/step - loss: 0.7192 - accuracy: 0.7686
----------------------------------------------------------------------------------------------------
max_words: 3000
max_len: 110
Test score: 0.7191842794418335
Test accuracy: 0.7686350345611572


In [96]:
df_w2v = data.copy()

In [97]:
df_w2v = df_w2v[df_w2v['Rating'] != 3]
df_w2v['target'] = (df_w2v['Rating'] > 3)*1
df_w2v = df_w2v.drop(['Rating'], axis=1)

In [98]:
df_w2v['target'] = df_w2v['target'].astype(int)
df_w2v['target'].value_counts()

1    16724
0     3024
Name: target, dtype: int64

In [99]:
df_train = df_w2v.loc[:14000]
df_val = df_w2v.loc[14001:]

In [100]:
df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_val['Content'] = df_val['Content'].apply(preprocess_text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [101]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_val['Content'] = df_val['Content'].apply(preprocess_text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [102]:
text_corpus_train = df_train['Content'].values
text_corpus_valid = df_val['Content'].values

In [103]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
# sequences_test = tokenizer.texts_to_sequences(text_corpus_test)
word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])
X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [104]:
y_train = df_train['target'].values
y_val = df_val['target'].values

## 1. Построить свёрточные архитектуры

In [105]:
results = []

In [106]:

num_classes = 1 
model = Sequential()
# model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=128,
              trainable=True,
              mask_zero=True))
    
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))
model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 100, 128)          1146112   
                                                                 
 masking_8 (Masking)         (None, 100, 128)          0         
                                                                 
 conv1d_7 (Conv1D)           (None, 98, 128)           49280     
                                                                 
 activation_7 (Activation)   (None, 98, 128)           0         
                                                                 
 global_max_pooling1d_3 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_18 (Dense)            (None, 10)                1290      
                                                     

In [107]:
early_stopping=EarlyStopping(monitor='val_loss', patience=5)  

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])


Epoch 1/10
24/24 [==============================] - 11s 415ms/step - loss: 0.4727 - accuracy: 0.8684 - val_loss: 0.3842 - val_accuracy: 0.8688
Epoch 2/10
24/24 [==============================] - 11s 461ms/step - loss: 0.3237 - accuracy: 0.8726 - val_loss: 0.2885 - val_accuracy: 0.8688
Epoch 3/10
24/24 [==============================] - 10s 407ms/step - loss: 0.2153 - accuracy: 0.8970 - val_loss: 0.2222 - val_accuracy: 0.8978
Epoch 4/10
24/24 [==============================] - 10s 401ms/step - loss: 0.1502 - accuracy: 0.9459 - val_loss: 0.2053 - val_accuracy: 0.9031
Epoch 5/10
24/24 [==============================] - 10s 404ms/step - loss: 0.1137 - accuracy: 0.9623 - val_loss: 0.2023 - val_accuracy: 0.9075
Epoch 6/10
24/24 [==============================] - 10s 401ms/step - loss: 0.0889 - accuracy: 0.9702 - val_loss: 0.2084 - val_accuracy: 0.9045
Epoch 7/10
24/24 [==============================] - 10s 402ms/step - loss: 0.0709 - accuracy: 0.9779 - val_loss: 0.2288 - val_accuracy: 0.9128

In [108]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
results.append(score[1])
print('-'*100)
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 111ms/step - loss: 0.3228 - accuracy: 0.8923
----------------------------------------------------------------------------------------------------
Test score: 0.3228277862071991
Test accuracy: 0.8923392295837402


## 2. Построить различные архитектуры с RNN

In [109]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))  

model.add(Dense(124, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# model.summary()

In [110]:
early_stopping=EarlyStopping(monitor='val_loss', patience=5)  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    # callbacks=[early_stopping])
                    callbacks=[early_stopping])

Epoch 1/10
24/24 [==============================] - 10s 363ms/step - loss: 0.4297 - accuracy: 0.8380 - val_loss: 0.2987 - val_accuracy: 0.8732
Epoch 2/10
24/24 [==============================] - 8s 342ms/step - loss: 0.2560 - accuracy: 0.8949 - val_loss: 0.2474 - val_accuracy: 0.8837
Epoch 3/10
24/24 [==============================] - 8s 344ms/step - loss: 0.1835 - accuracy: 0.9324 - val_loss: 0.2174 - val_accuracy: 0.8978
Epoch 4/10
24/24 [==============================] - 8s 346ms/step - loss: 0.1191 - accuracy: 0.9583 - val_loss: 0.2186 - val_accuracy: 0.9120
Epoch 5/10
24/24 [==============================] - 10s 405ms/step - loss: 0.0742 - accuracy: 0.9750 - val_loss: 0.2456 - val_accuracy: 0.9001
Epoch 6/10
24/24 [==============================] - 8s 349ms/step - loss: 0.0503 - accuracy: 0.9850 - val_loss: 0.2862 - val_accuracy: 0.9016
Epoch 7/10
24/24 [==============================] - 8s 340ms/step - loss: 0.0389 - accuracy: 0.9883 - val_loss: 0.3255 - val_accuracy: 0.8867
Epoc

In [111]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
results.append(score[1])
print('-'*100)
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 105ms/step - loss: 0.3855 - accuracy: 0.8887
----------------------------------------------------------------------------------------------------
Test score: 0.38549482822418213
Test accuracy: 0.8887137174606323


## LSTM

In [112]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [113]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])


Epoch 1/10
24/24 [==============================] - 7s 225ms/step - loss: 0.4517 - accuracy: 0.8504 - val_loss: 0.3781 - val_accuracy: 0.8688
Epoch 2/10
24/24 [==============================] - 5s 210ms/step - loss: 0.3383 - accuracy: 0.8816 - val_loss: 0.2970 - val_accuracy: 0.8770
Epoch 3/10
24/24 [==============================] - 6s 271ms/step - loss: 0.2591 - accuracy: 0.9018 - val_loss: 0.2600 - val_accuracy: 0.8814
Epoch 4/10
24/24 [==============================] - 5s 210ms/step - loss: 0.2025 - accuracy: 0.9279 - val_loss: 0.2396 - val_accuracy: 0.8963
Epoch 5/10
24/24 [==============================] - 5s 210ms/step - loss: 0.1514 - accuracy: 0.9479 - val_loss: 0.2501 - val_accuracy: 0.9008
Epoch 6/10
24/24 [==============================] - 5s 208ms/step - loss: 0.1147 - accuracy: 0.9599 - val_loss: 0.2464 - val_accuracy: 0.8971
Epoch 7/10
24/24 [==============================] - 5s 212ms/step - loss: 0.0862 - accuracy: 0.9718 - val_loss: 0.2640 - val_accuracy: 0.9016
Epoch 

In [114]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
results.append(score[1])
print('-'*100)
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 1s 71ms/step - loss: 0.3457 - accuracy: 0.8886
----------------------------------------------------------------------------------------------------
Test score: 0.3456871509552002
Test accuracy: 0.8885561227798462


## GRU

In [115]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
24/24 [==============================] - 25s 926ms/step - loss: 0.4950 - accuracy: 0.8530 - val_loss: 0.3281 - val_accuracy: 0.8688
Epoch 2/10
24/24 [==============================] - 22s 905ms/step - loss: 0.2558 - accuracy: 0.8802 - val_loss: 0.2377 - val_accuracy: 0.8792
Epoch 3/10
24/24 [==============================] - 23s 968ms/step - loss: 0.1828 - accuracy: 0.9271 - val_loss: 0.2255 - val_accuracy: 0.8889
Epoch 4/10
24/24 [==============================] - 22s 913ms/step - loss: 0.1353 - accuracy: 0.9509 - val_loss: 0.2428 - val_accuracy: 0.8941
Epoch 5/10
24/24 [==============================] - 22s 909ms/step - loss: 0.0952 - accuracy: 0.9673 - val_loss: 0.2661 - val_accuracy: 0.8956
Epoch 6/10
24/24 [==============================] - 23s 973ms/step - loss: 0.0652 - accuracy: 0.9787 - val_loss: 0.3031 - val_accuracy: 0.8911
Epoch 7/10
24/24 [==============================] - 22s 917ms/step - loss: 0.0479 - accuracy: 0.9855 - val_loss: 0.3594 - val_accuracy: 0.8949

In [116]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
results.append(score[1])
print('-'*100)
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 3s 208ms/step - loss: 0.4811 - accuracy: 0.8821
----------------------------------------------------------------------------------------------------
Test score: 0.4811377227306366
Test accuracy: 0.8820933103561401


## 3. Построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [117]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding="same"))

model.add(SimpleRNN(64))  

model.add(Dense(124, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 100, 128)          1146112   
                                                                 
 masking_12 (Masking)        (None, 100, 128)          0         
                                                                 
 conv1d_8 (Conv1D)           (None, 100, 64)           24640     
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 64)                8256      
                                                                 
 dense_26 (Dense)            (None, 124)               8060      
                                                                 
 dropout_9 (Dropout)         (None, 124)               0         
                                                                 
 dense_27 (Dense)            (None, 1)               

In [118]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
results.append(score[1])
print('-'*100)
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 2s 123ms/step - loss: 0.4844 - accuracy: 0.8811
----------------------------------------------------------------------------------------------------
Test score: 0.4843840003013611
Test accuracy: 0.881147563457489


## CNN + LSTM

In [119]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding="same"))
model.add(LSTM(64, recurrent_dropout=0.2))  # recurrent_dropout рвёт связи между RNN-ячейками
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
24/24 [==============================] - 27s 1s/step - loss: 0.4391 - accuracy: 0.8558 - val_loss: 0.3097 - val_accuracy: 0.8688
Epoch 2/10
24/24 [==============================] - 26s 1s/step - loss: 0.2644 - accuracy: 0.8854 - val_loss: 0.2404 - val_accuracy: 0.8874
Epoch 3/10
24/24 [==============================] - 24s 1s/step - loss: 0.1801 - accuracy: 0.9319 - val_loss: 0.2313 - val_accuracy: 0.8986
Epoch 4/10
24/24 [==============================] - 26s 1s/step - loss: 0.1161 - accuracy: 0.9586 - val_loss: 0.2540 - val_accuracy: 0.9008
Epoch 5/10
24/24 [==============================] - 24s 1s/step - loss: 0.0758 - accuracy: 0.9741 - val_loss: 0.3001 - val_accuracy: 0.8904
Epoch 6/10
24/24 [==============================] - 24s 1s/step - loss: 0.0543 - accuracy: 0.9822 - val_loss: 0.3309 - val_accuracy: 0.8949
Epoch 7/10
24/24 [==============================] - 26s 1s/step - loss: 0.0402 - accuracy: 0.9881 - val_loss: 0.3708 - val_accuracy: 0.8926
Epoch 8/10
24/24 [==

In [120]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
results.append(score[1])
print('-'*100)
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 3s 239ms/step - loss: 0.5089 - accuracy: 0.8837
----------------------------------------------------------------------------------------------------
Test score: 0.5089117288589478
Test accuracy: 0.8836696147918701


## LSTM + CNN

In [121]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True)
)

model.add(Masking(mask_value=0.0))

model.add(LSTM(64, recurrent_dropout=0.2, return_sequences=True))  
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding="same"))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
24/24 [==============================] - 27s 1s/step - loss: 0.4359 - accuracy: 0.8393 - val_loss: 0.3700 - val_accuracy: 0.8688
Epoch 2/10
24/24 [==============================] - 22s 918ms/step - loss: 0.3279 - accuracy: 0.8781 - val_loss: 0.2837 - val_accuracy: 0.8829
Epoch 3/10
24/24 [==============================] - 23s 976ms/step - loss: 0.2434 - accuracy: 0.9006 - val_loss: 0.2273 - val_accuracy: 0.8993
Epoch 4/10
24/24 [==============================] - 22s 914ms/step - loss: 0.1725 - accuracy: 0.9323 - val_loss: 0.2074 - val_accuracy: 0.9053
Epoch 5/10
24/24 [==============================] - 22s 914ms/step - loss: 0.1252 - accuracy: 0.9552 - val_loss: 0.2261 - val_accuracy: 0.9038
Epoch 6/10
24/24 [==============================] - 23s 979ms/step - loss: 0.0934 - accuracy: 0.9695 - val_loss: 0.2342 - val_accuracy: 0.9083
Epoch 7/10
24/24 [==============================] - 22s 916ms/step - loss: 0.0738 - accuracy: 0.9765 - val_loss: 0.2916 - val_accuracy: 0.9105
Ep

In [122]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
results.append(score[1])
print('-'*100)
print('Test score:', score[0])
print('Test accuracy:', score[1])

13/13 [==============================] - 3s 222ms/step - loss: 0.5498 - accuracy: 0.8796
----------------------------------------------------------------------------------------------------
Test score: 0.549797773361206
Test accuracy: 0.879571259021759


# Выводы


In [124]:
results

[0.8923392295837402,
 0.8887137174606323,
 0.8885561227798462,
 0.8820933103561401,
 0.881147563457489,
 0.8836696147918701,
 0.879571259021759]

###   Сверточная сеть показала лучшие результаты 2 и 3 место SimpleRNN и LSTM